# 1.데이터 준비와 전처리

In [5]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [7]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [8]:
print(ratings_cols)

['user_id', 'movie_id', 'rating', 'timestamp']


In [9]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head(10)

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


# 2.분석해 봅시다.

In [10]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [11]:
#rating에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [12]:
#가장 인기있는 영화 30개(인기순)
top_30_movie_count = ratings.groupby('movie_id')['user_id'].count()
top_30_movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

# 3.내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

In [13]:
ratings
my_ratings = pd.DataFrame({'user_id': ['6041']*5, 'movie_id' : [17, 25, 35, 7, 46] , 'count':[4, 5, 4, 3, 5], 'timestamp' : [956715570, 956715571, 956715572, 956715573, 956715574],})
data = ratings.append(my_ratings)
data.tail(10)


,user_id,movie_id,count,timestamp
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648
1000208,6040,1097,4,956715569
0,6041,17,4,956715570
1,6041,25,5,956715571
2,6041,35,4,956715572
3,6041,7,3,956715573
4,6041,46,5,956715574


# 4.  CSR matrix를 직접 만들어 봅시다.

In [14]:
# 고유한 유저, 아티스트를 idx로 만들기
user_idx = data['user_id'].unique()
movie_idx = data['movie_id'].unique()
len(data)


836483

In [15]:
data.user_id = data.user_id.astype(int)

In [16]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

#num_user = data['user_id'].nunique()
#num_artist = data['movie_id'].nunique()

csr_data = csr_matrix((data['count'], (data.user_id, data.movie_id)))
csr_data

<6042x3953 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

# 5. als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [17]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [18]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [19]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3953x6042 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [20]:
# 모델 훈련
als_model.fit(csr_data_transpose)

# 6. 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [24]:
als_model.user_factors[6041]

array([ 0.4928025 ,  0.3361755 ,  0.41774473, -0.21215333, -0.33994597,
       -0.2004896 , -0.34932312,  0.23973456,  0.00880986, -0.09545758,
       -0.3965575 , -0.04570492,  0.1982317 , -0.8024196 ,  0.14842261,
        0.43035546, -0.7222505 ,  0.12927599, -0.01924999, -0.2329145 ,
        0.33149853, -0.12472852, -0.14227492, -0.00150261, -0.266229  ,
       -0.17582928, -0.11120012,  0.40560168,  0.46553805,  0.3662627 ,
       -0.18041092,  0.18299162, -0.38107684,  0.7190276 ,  0.03980404,
        0.2137638 ,  0.27752757,  0.03298655,  0.26968175, -0.88627756,
       -0.12789178,  0.09133136,  0.70694244, -0.37315065, -0.02150708,
        0.09498032, -0.03294109,  0.52792215,  0.10116234, -0.11289696,
        0.25437787,  0.08749759,  0.15394041,  0.01355938,  0.12380273,
        0.25377086, -0.45194396, -0.18995963, -0.8336682 , -0.25845304,
        0.3344501 , -0.02841464,  0.11658666, -0.46421778,  0.08800514,
        0.0759114 , -0.6285899 , -0.12657425,  0.15149389,  0.75

In [28]:
my_ratings

,user_id,movie_id,count,timestamp
0,6041,17,4,956715570
1,6041,25,5,956715571
2,6041,35,4,956715572
3,6041,7,3,956715573
4,6041,46,5,956715574


In [31]:
def checkMovie(user_id, movie_id):
    user_factor, movie_factor = als_model.user_factors[user_id], als_model.item_factors[movie_id]
    
    return np.dot(user_factor, movie_factor)

checkMovie(data.iloc[0]['user_id'], data.iloc[0]['movie_id'])

0.7572931

In [32]:
checkMovie(88,1)

0.2347305

# 7. 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [37]:
def similar_movie(movie_ids):
    return movies[movies['movie_id'].isin(movie_ids)]

similar_movie_ids = [i[0] for i in als_model.similar_items(my_ratings.iloc[0]['movie_id'])]
get_movies_by_ids(similar_movie_ids)

,movie_id,title,genre
16,17,Sense and Sensibility (1995),Drama|Romance
57,58,"Postino, Il (The Postman) (1994)",Drama|Romance
262,265,Like Water for Chocolate (Como agua para choco...,Drama|Romance
505,509,"Piano, The (1993)",Drama|Romance
530,534,Shadowlands (1993),Drama|Romance
827,838,Emma (1996),Comedy|Drama|Romance
1166,1183,"English Patient, The (1996)",Drama|Romance|War
1336,1357,Shine (1996),Drama|Romance
1597,1643,"Mrs. Brown (Her Majesty, Mrs. Brown) (1997)",Drama|Romance
3191,3260,Howards End (1992),Drama


# 8. 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [40]:
recommend_movie = [i[0] for i in als_model.recommend(data.iloc[0]['user_id'], csr_data)]
get_movies_by_ids(recommend_movie)

,movie_id,title,genre
315,318,"Shawshank Redemption, The (1994)",Drama
360,364,"Lion King, The (1994)",Animation|Children's|Musical
908,920,Gone with the Wind (1939),Drama|Romance|War
1207,1225,Amadeus (1984),Drama
1262,1282,Fantasia (1940),Animation|Children's|Musical
1656,1704,Good Will Hunting (1997),Drama
2009,2078,"Jungle Book, The (1967)",Animation|Children's|Comedy|Musical
2012,2081,"Little Mermaid, The (1989)",Animation|Children's|Comedy|Musical|Romance
2027,2096,Sleeping Beauty (1959),Animation|Children's|Musical
2789,2858,American Beauty (1999),Comedy|Drama
